In [1]:
# load imports and workspace

import azureml.core
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies
import os

ws = Workspace.from_config()

print(ws.name, ws.location, ws.resource_group, sep='\t')
print("Azure ML SDK Version: ", azureml.core.VERSION)

OpenClassrooms	francecentral	Calculators
Azure ML SDK Version:  1.34.0


In [2]:
# create environment

def reg_env():
    env = Environment('dl-env')
    cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]',
                                                'azureml-defaults',
                                                'tensorflow==2.6.0',
                                                'joblib==1.0.1',
                                                'Unidecode==1.3.2',
                                                'nltk==3.6.5'],
                                conda_packages=['pip==21.3'])
    env.python.conda_dependencies = cd
    env.register(workspace=ws)

# reg_env()

In [3]:
# model register

def reg_model():
    model = Model.register(model_path="deploy/glove_lstm_nn.h5",
                        model_name="twitter-sentiment-glove-lstm",
                        description="Predict twits sentiments with Glove LSTM RNN",
                        workspace=ws)

# reg_model()


In [4]:
# model deploy

model = Model(ws, 'twitter-sentiment-glove-lstm')

myenv = Environment.get(workspace=ws, name="dl-env", version="11") # CARE UPDATING VERSION ON NEW DEPLOY

inference_config = InferenceConfig(source_directory="deploy", entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "twitter-sentiment",  "method": "glove-lstm"}, 
                                               description='Predict twits sentiments with Glove LSTM RNN')

service = Model.deploy(workspace=ws, 
                       name='glove-lstm-service', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-29 14:38:17+00:00 Creating Container Registry if not exists.
2021-10-29 14:38:17+00:00 Registering the environment.
2021-10-29 14:38:17+00:00 Use the existing image.
2021-10-29 14:38:18+00:00 Generating deployment configuration.
2021-10-29 14:38:19+00:00 Submitting deployment to compute..
2021-10-29 14:39:02+00:00 Checking the status of deployment glove-lstm-service..
2021-10-29 14:41:21+00:00 Checking the status of inference endpoint glove-lstm-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [5]:
print(service.scoring_uri)

http://818544a3-f7cf-4321-a04e-d05e995e6dd0.francecentral.azurecontainer.io/score


In [6]:
# service.delete()